In [1]:
from selenium import webdriver
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.chrome.options import Options
import requests
from bs4 import BeautifulSoup

from selenium.common.exceptions import NoSuchElementException

from PIL import Image
import io

import time

import pandas as pd

import utils.functions as F
import utils.data as data

from absl import app
from absl import flags
from absl import logging
import bs4

In [2]:
url = 'https://www.jalan.net/kankou/060000/?screenId=OUW2202&influxKbn=0'

options = Options()
options.add_argument('--headless')
browser = webdriver.Chrome(ChromeDriverManager().install(), options=options)
browser.get(url)

url = browser.current_url
res = requests.get(url)
soup = BeautifulSoup(res.content, 'html.parser')
a_elems = soup.find_all('p', attrs={'class' : 'item-name'})

[WDM] - ====== WebDriver manager ======
[WDM] - Current google-chrome version is 102.0.5005
[WDM] - Get LATEST chromedriver version for 102.0.5005 google-chrome
[WDM] - Driver [/Users/m.kazu/.wdm/drivers/chromedriver/mac64/102.0.5005.61/chromedriver] found in cache
Some characters could not be decoded, and were replaced with REPLACEMENT CHARACTER.


In [3]:
res = requests.get(url)
soup = BeautifulSoup(res.content, 'html.parser')

Some characters could not be decoded, and were replaced with REPLACEMENT CHARACTER.


In [4]:
def get_urls(soup:bs4.BeautifulSoup, target:str='suumo') -> list:
    
    print(target)

    h2_elems = soup.find_all('p', attrs={'class' : 'item-name'})

    urls = []

    for h2_elem in h2_elems:
        a_elem = h2_elem.find('a')
        url = a_elem.attrs['href']

        urls.append(url)

    return urls

In [5]:
urls = get_urls(soup)

suumo


In [6]:
urls

['//www.jalan.net/kankou/spt_guide000000152086/',
 '//www.jalan.net/kankou/spt_06203cc3320040667/',
 '//www.jalan.net/kankou/spt_06424ae2180021220/',
 '//www.jalan.net/kankou/spt_06424ab2010001853/',
 '//www.jalan.net/kankou/spt_06212cd2110077654/',
 '//www.jalan.net/kankou/spt_06201cd2110077451/',
 '//www.jalan.net/kankou/spt_06204ag2130014968/',
 '//www.jalan.net/kankou/spt_alikejpB000375810/',
 '//www.jalan.net/kankou/spt_06201ae3292019427/',
 '//www.jalan.net/kankou/spt_06204ae2180022287/',
 '//www.jalan.net/kankou/spt_guide000000186569/',
 '//www.jalan.net/kankou/spt_06441ab2010001861/',
 '//www.jalan.net/kankou/spt_06441ag2130009280/',
 '//www.jalan.net/kankou/spt_06382ag2130011725/',
 '//www.jalan.net/kankou/spt_guide000000180653/',
 '//www.jalan.net/kankou/spt_guide000000161658/',
 '//www.jalan.net/kankou/spt_06323ag2130015562/',
 '//www.jalan.net/kankou/spt_06210cb3512010894/',
 '//www.jalan.net/kankou/spt_06207cd2110078131/',
 '//www.jalan.net/kankou/spt_06202ag2130014416/',


In [7]:
for url in urls:
  page_url = "https:" + url + 'kuchikomi'
  page_res = requests.get(page_url)
  page_soup = BeautifulSoup(page_res.content, 'html.parser')
  break

In [8]:
all_content = page_soup.find_all('div', attrs={'class' : 'item-listContents'})

In [9]:
review_id = 0
for content in all_content:
  if content.find('picture', attrs={'class' : 'item-mainImg'}):
    print('写真ある時こっち')
    review_property_dict = {}

    div_elem = content.find('p', attrs={'class' : 'item-title'})
    a_elem = div_elem.find('a')
    review_page_url = 'https:' + a_elem.attrs['href']
    review_page_res = requests.get(review_page_url)
    review_page_soup = BeautifulSoup(review_page_res.content, 'html.parser')

    review_soup = review_page_soup.find('p', attrs={'class' : 'reviewText'})

    title = review_page_soup.find('h1', attrs={'class' : 'basicTitle'})
    review = review_soup.text.replace('\n', '')
    review_property_dict['title'] = title
    review_property_dict['review'] = review

    review_properties = review_page_soup.find('ul', attrs={'class' : 'reviewDetail'})
    review_properties = review_properties.find_all('li')
    review_properties=[review_property.text.strip() for review_property in review_properties]

    for review_property in review_properties:
      column_name = review_property.split('：')[0]
      column_data = review_property.split('：')[1]

      review_property_dict[column_name] = column_data

    img_urls = []
    img_contents_block = review_page_soup.find('ul', attrs={'class' : 'cassetteList-photo'})
    img_contents = img_contents_block.find_all('li', attrs={'class' : 'lightbox'})
    # print(img_contents)

    img_id = 0

    img_name_list = list()

    for img_content in img_contents:
      img_elem = img_content.find('source')
      img_url = img_elem.attrs['srcset']
      img_url = 'https:' + img_url

      img = Image.open(io.BytesIO(requests.get(img_url).content))
      img_name=str(review_id) + '_' + str(img_id)
      img.save(f'imgs/jalan/{img_name}.jpg')

      img_id += 1

    reviews_property_dict = {}

    reviews_property_dict[review_id] = {
                                        "レビューID" : review_id,
                                        "レビューURL" : review_page_url,
                                        "タイトル" : title,
                                        "レビュー" : review,
                                        "行った時期" : review_property_dict['行った時期'],
                                        "混雑具合" : review_property_dict['混雑具合'],
                                        "滞在時間" : review_property_dict['滞在時間'],
                                        "投稿日" : review_property_dict['投稿日'],}
    
  else:
    print('写真ない時こっち')

写真ない時こっち
写真ない時こっち
写真ない時こっち
写真ない時こっち
写真ない時こっち
写真ない時こっち
写真ある時こっち


Some characters could not be decoded, and were replaced with REPLACEMENT CHARACTER.


写真ない時こっち
写真ない時こっち
写真ない時こっち


In [10]:

def jalan():
    print('jalan')
    review_id = 0
    pref_sum_count = 0

    for url in [data.jalan_urls['Yamagata']]:
        prefecture_name = 'Yamagata'
        pref_sum_count += 1
        url = url
        data_list = [] #多分いらない
        page_count = 0

        logging.info(pref_sum_count, prefecture_name)

        options = Options()
        options.add_argument('--headless')
        browser = webdriver.Chrome(ChromeDriverManager().install(), options=options)
        browser.get(url)

        # 次へをクリックして、観光地自体のページがなくなるまで繰り返しページを探索する
        while True:
            url = browser.current_url
            logging.info(url)
            #ブラウザの起動
            options = Options()
            options.add_argument('--headless')
            browser = webdriver.Chrome(ChromeDriverManager().install(), options=options)

            browser.get(url)

            try:
                res = requests.get(url)
                soup = BeautifulSoup(res.content, 'html.parser')
                urls = F.get_urls(soup, target='jalan') #1ページに含まれる全観光地のリンク
                
                for url in urls:
                    # 一つの観光地について
                    page_url = "https:" + url + 'kuchikomi'
                    while True:
                        page_res = requests.get(page_url)
                        page_soup = BeautifulSoup(page_res.content, 'html.parser')
                        
                        try:
                            # 観光地のレビュー、一覧ページ
                            all_content = page_soup.find_all('div', attrs={'class' : 'item-listContents'})
                            reviews_dict = {}
                            reviews_img_dict = {}
                            # print('all', all_content)
                            for content in all_content:
                                print(type(content))
                                print(F.is_existing_img(content))
                                # print(content)
                                if F.is_existing_img(content):
                                    print('existing content')
                                #     # 写真ある場合
                                #     review_page_soup = F.get_review_page_soup(content)
                                #     review_property_dict = F.get_jalan_review(review_id, review_page_soup)
                                #     img_name_list = F.get_review_img(review_id, review_page_soup)
                                # else:
                                #     print('None')

                                # reviews_dict[review_id] = {
                                #                 "レビューID" : review_property_dict['review_id'],
                                #                 # "レビューURL" : review_property_dict['review_page_url'],
                                #                 "タイトル" : review_property_dict['title'],
                                #                 "レビュー" : review_property_dict['review'],
                                #                 "行った時期" : review_property_dict['行った時期'],
                                #                 "混雑具合" : review_property_dict['混雑具合'],
                                #                 "滞在時間" : review_property_dict['滞在時間'],
                                #                 "投稿日" : review_property_dict['投稿日'],
                                #                 }
                                # reviews_img_dict[review_id] = {"imgs" : img_name_list}
                                else:
                                    print('none photo')

                            # attribute_df = pd.DataFrame(reviews_dict).transpose()
                            # attribute_df.to_csv('csv/attribute_{filename}_{page_num}.csv'.format(filename = prefecture_name, page_num=page_count))
                            # imgs_df = pd.DataFrame(reviews_img_dict)
                            # imgs_df.to_csv('csv/imgs_{filename}_{page_num}.csv'.format(filename = prefecture_name, page_num=page_count))

                            browser.find_element_by_link_text('次へ').click()
                            if page_count % 10 == 0:
                                logging.info("pages:{0}".format(page_count))
                                logging.info('==============================================')
                                time.sleep(60)
                            time.sleep(60)

                            page_count += 1
                        except NoSuchElementException:
                            browser.quit()
                        break

            except NoSuchElementException:
                browser.quit()
                break

        logging.info('browser quit')

In [ ]:
jalan()

In [21]:
def jalan():
    print('jalan')
    pref_sum_count = 0

    for url in [data.jalan_urls['Yamagata']]:
        prefecture_name = 'Yamagata'
        pref_sum_count += 1
        landmark_count = 0
        url = url
        data_list = [] #多分いらない
        page_count = 0

        logging.info(pref_sum_count, prefecture_name)

        options = Options()
        options.add_argument('--headless')
        browser = webdriver.Chrome(ChromeDriverManager().install(), options=options)
        browser.get(url)

        # 次へをクリックして、観光地自体のページがなくなるまで繰り返しページを探索する
        while True:
            url = browser.current_url
            logging.info(url)
            #ブラウザの起動
            # ⓪任意の県だけのページを取得
            options = Options()
            options.add_argument('--headless')
            browser = webdriver.Chrome(ChromeDriverManager().install(), options=options)
            browser.get(url)

            try:
                res = requests.get(url)
                soup = BeautifulSoup(res.content, 'html.parser')
                urls = F.get_urls(soup, target='jalan') #⓪任意の件に含まれる1ページの全観光地のリンク

                for url in urls:
                    # ①一つの観光地についての口コミ１ページ
                    page_url = "https:" + url + 'kuchikomi'
                    review_count = 0
                    while True:
                        options = Options()
                        options.add_argument('--headless')
                        browser_page = webdriver.Chrome(ChromeDriverManager().install(), options=options)
                        browser_page.get(page_url)

                        page_res = requests.get(page_url)

                        page_current_url = browser_page.current_url

                        page_soup = BeautifulSoup(page_res.content, 'html.parser')

                        # ①観光地のレビュー、一覧ページ
                        all_content = page_soup.find_all('div', attrs={'class' : 'item-listContents'})
                        reviews_dict = {}
                        reviews_img_dict = {}
                        for content in all_content:

                            if F.is_existing_img(content):
                                # ②
                                review_page_soup = F.get_review_page_soup(content)
                                review_property_dict = F.get_jalan_review(review_count, content, review_page_soup)
                                print('existing content url', review_property_dict['review_page_url'])
                                img_name_list = F.get_review_img(landmark_count, review_count, review_page_soup)

                                review_count += 1
                            page_count += 1

                        try:
                            browser_page.find_element_by_link_text('次へ').click()
                            page_current_url = browser_page.current_url
                            page_url = browser_page.current_url
                        except NoSuchElementException:
                            browser.quit()
                            break

                    reviews_dict[review_id] = {
                                            "レビューID" : review_property_dict['review_id'],
                                            "レビューURL" : review_property_dict['review_page_url'],
                                            "タイトル" : review_property_dict['title'],
                                            "レビュー" : review_property_dict['review'],
                                            "行った時期" : review_property_dict['行った時期'],
                                            "混雑具合" : review_property_dict['混雑具合'],
                                            "滞在時間" : review_property_dict['滞在時間'],
                                            "投稿日" : review_property_dict['投稿日'],
                                            }
                    reviews_img_dict[review_id] = {"imgs" : img_name_list}

                    attribute_df = pd.DataFrame(reviews_dict).transpose()
                    attribute_df.to_csv('csv/jalan/attribute_{filename}_{landmark_count}.csv'.format(filename = prefecture_name, landmark_count=landmark_count))
                    imgs_df = pd.DataFrame(reviews_img_dict)
                    imgs_df.to_csv('csv/jalan/imgs_{filename}_{landmark_count}.csv'.format(filename = prefecture_name, landmark_count=landmark_count))

                    landmark_count += 1

            except NoSuchElementException:
                browser.quit()
                break

        logging.info('browser quit')

In [23]:
jalan()

jalan



[WDM] - ====== WebDriver manager ======
INFO:WDM:====== WebDriver manager ======
[WDM] - Current google-chrome version is 102.0.5005
INFO:WDM:Current google-chrome version is 102.0.5005
[WDM] - Get LATEST chromedriver version for 102.0.5005 google-chrome
INFO:WDM:Get LATEST chromedriver version for 102.0.5005 google-chrome
[WDM] - Driver [/Users/m.kazu/.wdm/drivers/chromedriver/mac64/102.0.5005.61/chromedriver] found in cache
INFO:WDM:Driver [/Users/m.kazu/.wdm/drivers/chromedriver/mac64/102.0.5005.61/chromedriver] found in cache


[WDM] - ====== WebDriver manager ======
INFO:WDM:====== WebDriver manager ======
[WDM] - Current google-chrome version is 102.0.5005
INFO:WDM:Current google-chrome version is 102.0.5005
[WDM] - Get LATEST chromedriver version for 102.0.5005 google-chrome
INFO:WDM:Get LATEST chromedriver version for 102.0.5005 google-chrome
[WDM] - Driver [/Users/m.kazu/.wdm/drivers/chromedriver/mac64/102.0.5005.61/chromedriver] found in cache
INFO:WDM:Driver [/Users/m.kazu/.wdm/drivers/chromedriver/mac64/102.0.5005.61/chromedriver] found in cache


[WDM] - ====== WebDriver manager ======
INFO:WDM:====== WebDriver manager ======
[WDM] - Current google-chrome version is 102.0.5005
INFO:WDM:Current google-chrome version is 102.0.5005
[WDM] - Get LATEST chromedriver version for 102.0.5005 google-chrome
INFO:WDM:Get LATEST chromedriver version for 102.0.5005 google-chrome
[WDM] - Driver [/Users/m.kazu/.wdm/drivers/chromedriver/mac64/102.0.5005.61/chromedriver] found in cache
INFO:WDM:Driver [/Users/m.kazu/.wdm/drivers/chromedriver/mac64/102.0.5005.61/chromedriver] found in cache


page url :  https://www.jalan.net/kankou/spt_guide000000152086/kuchikomi/


...

[WDM] - ====== WebDriver manager ======
INFO:WDM:====== WebDriver manager ======
[WDM] - Current google-chrome version is 102.0.5005
INFO:WDM:Current google-chrome version is 102.0.5005
[WDM] - Get LATEST chromedriver version for 102.0.5005 google-chrome
INFO:WDM:Get LATEST chromedriver version for 102.0.5005 google-chrome
[WDM] - Driver [/Users/m.kazu/.wdm/drivers/chromedriver/mac64/102.0.5005.61/chromedriver] found in cache
INFO:WDM:Driver [/Users/m.kazu/.wdm/drivers/chromedriver/mac64/102.0.5005.61/chromedriver] found in cache


page url :  https://www.jalan.net/kankou/spt_guide000000152086/kuchikomi/page_3/?screenId=OUW2202&reviewRefineCompanion=all&reviewRefineMonth=all&resultSort=pd
aftre current page url :  https://www.jalan.net/kankou/spt_guide000000152086/kuchikomi/page_4/?screenId=OUW2202&reviewRefineCompanion=all&reviewRefineMonth=all&resultSort=pd
次へをクリックしました



[WDM] - ====== WebDriver manager ======
INFO:WDM:====== WebDriver manager ======
[WDM] - Current google-chrome version is 102.0.5005
INFO:WDM:Current google-chrome version is 102.0.5005
[WDM] - Get LATEST chromedriver version for 102.0.5005 google-chrome
INFO:WDM:Get LATEST chromedriver version for 102.0.5005 google-chrome
[WDM] - Driver [/Users/m.kazu/.wdm/drivers/chromedriver/mac64/102.0.5005.61/chromedriver] found in cache
INFO:WDM:Driver [/Users/m.kazu/.wdm/drivers/chromedriver/mac64/102.0.5005.61/chromedriver] found in cache


page url :  https://www.jalan.net/kankou/spt_guide000000152086/kuchikomi/page_4/?screenId=OUW2202&reviewRefineCompanion=all&reviewRefineMonth=all&resultSort=pd


AttributeError: 'NoneType' object has no attribute 'text'